# Getting the Data and Environment

In [2]:
# Download gutenberg dataset
# This is the dataset on which all your models will be trained
# https://drive.google.com/file/d/0B2Mzhc7popBga2RkcWZNcjlRTGM/edit

# Step 1: Get the corpus and load it in

import os
def get_corpus (n):
    files = [f for f in os.listdir("./Gutenberg") if os.path.isfile(os.path.join("./Gutenberg", f))]
    count = 0
    for f in files:
        count += 1
    # print ("There are " + str(count) + " files in this corpus.")
    if n > count:
        return 0
    else:
        return files [n-1]

# Part 1a: Split into Sentences and Tokenization

In [3]:
import sys, re

def clean(text):
    # text is stripped of all extra white spaces here
    cleaned_text = re.sub(r'[\s]+', ' ',text)
    return cleaned_text


caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"

def replace (text):
    text = re.sub(prefixes,"\\1<prd>",text)
    if "Ph.D" in text: 
        text = text.replace("Ph.D.","Ph<prd>D<prd>")
        
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    
    if "”" in text: 
        text = text.replace(".”","”.")
    if "\"" in text: 
        text = text.replace(".\"","\".")
    if "!" in text: 
        text = text.replace("!\"","\"!")
    if "?" in text: 
        text = text.replace("?\"","\"?")
        
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    return text

def sentencizer(text):   
    text = " " + text + "  "
    text = text.replace("\n"," ")
    
    text = replace(text)
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def tokenize(sentences):
    # sentences is a list of sentences, which is a list of words
    tokenized_sentences = []
    for line in sentences:
        for tokens in line:
            tokenized_sentences.append(re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\w]+",tokens))
    return tokenized_sentences
    
def characterizer(word):
    characterized_words = []
    for ch in word:
        characterized_words.append(str(ch))
    return characterized_words

# Part 1b: Initial Language Model

In [4]:
def get_and_sort(n, size, corpus, loc):
    tokens = get_ngrams(n, corpus)
    sorted_tokens = sorted(tokens.items(), key = lambda x: x[1], reverse=True)
    if (loc == 't'):
        sorted_tokens = sorted_tokens[:size]
    if (loc == 'b'):
        sorted_tokens = sorted_tokens[-size:]
    return (sorted_tokens)

def get_unigrams(corpus):
    tokens = {}
    for sentence in corpus:
        for word in sentence:
            if word not in tokens: 
                tokens[word] = 0
            tokens[word] += 1
    return tokens

def get_bigrams(corpus):
    bigrams = {}
    for sentence in corpus:
        for index, word in enumerate(sentence):
            if index > 0:
                pair  = (sentence[index - 1], word)
                if pair not in bigrams:
                    bigrams[pair] = 0
                bigrams[pair] += 1
    return bigrams

def get_trigrams(corpus):
    trigrams = {}
    for sentence in corpus:
        for index, word in enumerate(sentence):
            if index > 1:
                three = (sentence[index - 2], sentence[index - 1], word)
                if three not in trigrams:
                    trigrams[three] = 0
                trigrams[three] += 1
    return trigrams

def get_fourgrams(corpus):
    fourgrams = {}
    for sentence in corpus:
        for index, word in enumerate(sentence):
            if index > 2:
                four = (sentence[index - 3], sentence[index - 2], sentence[index - 1], word)
                if four not in fourgrams:
                    fourgrams[four] = 0
                fourgrams[four] += 1
    return fourgrams

def get_ngrams(n, corpus):
    ngrams = {}
    if n == 1:
        ngrams = get_unigrams(corpus)
    elif n == 2:
        ngrams = get_bigrams(corpus)
    elif n == 3:
        ngrams = get_trigrams(corpus)
    elif n == 4:
        ngrams = get_fourgrams(corpus)
    return ngrams

def get_frequencies(count_ngrams):
    total = sum([count for (_, count) in count_ngrams])
    return [(gram, count / total) for (gram, count) in count_ngrams]

# Part 1c: Zipf's Law Implementation

In [5]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)

def get_curves(m, tab_size, corpus):
    graphable_ngrams = get_and_sort(m, tab_size*100, corpus, 't')
    if m == 1:
        iplot([{"x" : list(zip(*graphable_ngrams))[0], "y": list(zip(*graphable_ngrams))[1]}])
    else:
        iplot([{"x" : ['_'.join(x) for x in list(zip(*graphable_ngrams))[0]], "y": list(zip(*graphable_ngrams))[1]}])
        
def get_tables(n, tab_size, corpus):
    top_sorted_nchars = get_and_sort(n, tab_size, corpus, 't')
    bottom_sorted_nchars = get_and_sort(n, tab_size, corpus, 'b')
    top_frequency_nchars = get_frequencies(top_sorted_nchars)
    bottom_frequency_nchars = get_frequencies(bottom_sorted_nchars)
    print(top_sorted_nchars)
    print(bottom_sorted_nchars)
    
from ipy_table import *

# Part 1 Executable

In [17]:
file = get_corpus(1061)
f = open("./Gutenberg/" + file, "+r")
    
text = f.read()
    
# Step one, clean the corpus. 
cleaned_text = clean(text)

# Step two, split into sentences.
split_text = sentencizer(cleaned_text)
    
# Step three, tokenize
untokenized_text = []
for sentence in split_text:
    untokenized_text.append([sentence])
    
corpus = tokenize (untokenized_text)

characterized_corpus = []
for sentence in corpus:
    for word in sentence:
        characterized_corpus.append(characterizer(word))

# Step four, get ngrams, frequencies and the Zipf's curves
unigrams_sorted = get_and_sort(1, 1000, corpus, 't')
bigrams_sorted = get_and_sort(2, 1000, corpus, 't')
trigrams_sorted = get_and_sort(3, 1000, corpus, 't')
fourgrams_sorted = get_and_sort(4, 1000, corpus, 't')

m = 1
n = 3
tab = 5
get_tables(m, tab, corpus)
get_curves(m, tab, corpus)
get_tables(n, tab, characterized_corpus)
get_curves(n, tab, characterized_corpus)

[('the', 5143), ('to', 2603), ('of', 2577), ('and', 2494), ('a', 1935)]
[('perennial', 1), ('obliterate', 1), ('legend', 1), ('Romance', 1), ('END', 1)]


[(('t', 'h', 'e'), 7126), (('a', 'n', 'd'), 2953), (('i', 'n', 'g'), 2635), (('h', 'e', 'r'), 1810), (('h', 'a', 't'), 1774)]
[(('M', 'e', 'n'), 1), (('T', 'i', 'm'), 1), (('S', 'a', 'x'), 1), (('a', 'x', 'o'), 1), (('x', 'o', 'n'), 1)]


# Part 2a: Smoothing and Backoff

In [7]:
import math 
def laplace_smoothing(n, n_grams):
    orig_ngram_count = get_and_sort(n, 100000, n_grams, 't')
    orig_lessngram_count = get_and_sort(1, 100000, n_grams, 't')
    
    total_ngram = sum([count for (_, count) in orig_ngram_count])
    total_lessngram = sum([count for (_, count) in orig_lessngram_count])
    
    enhanced_probs = list()
    
    for (gram, orig_freq) in orig_ngram_count:
        enhanced_probs.append((gram, (orig_freq + 1)/(total_ngram + 100000)))
    
    return enhanced_probs

def eq_classes(ngrams):
    eq_class = dict()
    for gram, freq in ngrams:
        if freq not in eq_class:
            eq_class[freq] = list()
        eq_class[freq].append(gram)
    return eq_class

def heldout(corpus, n):
    length = int(len(corpus)/2)
    #print(length)
    training = corpus[:length]
    heldout = corpus[-length:]
    utraining = get_and_sort(n, 1000, training, 't')
    uheldout = get_and_sort(n, 1000, heldout, 't')
    tr_eq_cls = eq_classes(utraining)
    hest = {}
    for r in tr_eq_cls:
        t_r = sum([uheldout[u][0] for u in tr_eq_cls[r] if u in utraining])
        hest[r] = t_r
        
    #N = sum(utraining[i][1] for i in )
    print (tr_eq_cls)
    return hest

def good_turing(n_grams):
    unigram_count = get_and_sort(1, 1000, n_grams, 't')
    # unigram_count is now a list of (gram, frequency)
    
    high_freq = unigram_count[0][1]
    
    # Good Turing first counts the number of ngrams of a particular freq 
    nrs = eq_classes(n_grams)
    nr_counts = {k : len(v) for k, v in nrs.items()}
    nr_probs = {k : (k*v)/float(N) for k, v in nr_counts.items()} # P = r * Nr / N
    sorted_nrs = sorted(nr_counts.items(), reverse = True)
    #sorted_probs = sorted(nr_probs.items())
    MAX = sorted_nrs[0][1]
    new_nrs = {}
    for r, nr in nrs.items():
        if (r+1) in nrs:
            new_nr = ((r+1) * nrs[r+1]) / float(N) 
        else:
            new_nr = MAX*r**-2 / float(N)
        new_nrs[r] = new_nr
    return new_nrs
          


# only 1 of the next 2 have to be implemented

def backoff(n_grams):
  # perform Backoff
    pass
  
def deleted_interpolation(n_grams):
  # perform deleted Interpolation
    pass
  
  

# Part 2 Executable

In [9]:
n = 2
lap_probs = laplace_smoothing(2, corpus)
print(lap_probs[:5])
print(lap_probs[-5:])

lap_probs = laplace_smoothing(n, characterized_corpus)
print(lap_probs[:5])
print(lap_probs[-5:])


eq_classes(bigrams_sorted)
# good_turing(bigrams_sorted)
# heldout(corpus, 2)

[(('of', 'the'), 0.003372745187206306), (('in', 'the'), 0.0017756989107603023), (('to', 'the'), 0.0017756989107603023), (('at', 'the'), 0.0011206392516078737), (('with', 'a'), 0.001082743238268477)]
[(('is', 'founded'), 1.0827432382684771e-05), (('founded', 'this'), 1.0827432382684771e-05), (('this', 'Romance'), 1.0827432382684771e-05), (('Romance', 'of'), 1.0827432382684771e-05), (('THE', 'END'), 1.0827432382684771e-05)]
[(('t', 'h'), 0.029463583737035892), (('h', 'e'), 0.028163605273605247), (('i', 'n'), 0.0171488176942378), (('a', 'n'), 0.014707141578293214), (('e', 'r'), 0.014670814834804135)]
[(('O', 'w'), 5.189534784154274e-06), (('A', 'S'), 5.189534784154274e-06), (('f', 'n'), 5.189534784154274e-06), (('i', 'h'), 5.189534784154274e-06), (('x', 'o'), 5.189534784154274e-06)]


{622: [('of', 'the')],
 327: [('in', 'the'), ('to', 'the')],
 206: [('at', 'the')],
 199: [('with', 'a')],
 169: [('the', 'prince')],
 166: [('on', 'the')],
 163: [('to', 'be')],
 161: [('for', 'the')],
 160: [('and', 'the')],
 132: [('of', 'his')],
 128: [('in', 'a')],
 122: [('of', 'a')],
 120: [('I', 'have')],
 117: [('that', 'the')],
 106: [('by', 'the')],
 105: [('it', 'was')],
 103: [('the', 'captain')],
 102: [('I', 'am')],
 97: [('had', 'been')],
 95: [('did', 'not')],
 94: [('from', 'the'), ('said', 'the')],
 92: [('the', 'king')],
 91: [('with', 'the')],
 90: [('that', 'I')],
 89: [('he', 'had')],
 88: [('it', 'is'), ('the', 'Hebrew')],
 87: [('was', 'a')],
 86: [('to', 'his')],
 84: [('returned', 'the'), ('that', 'he')],
 82: [('into', 'the')],
 79: [('have', 'been')],
 77: [('he', 'was')],
 76: [('as', 'he')],
 74: [('a', 'few')],
 72: [('I', 'will')],
 69: [('one', 'of'), ('and', 'a'), ('and', 'I')],
 66: [('the', 'chief'), ('for', 'a')],
 65: [('the', 'same')],
 64: [('is

# Part 3: Grammatical Score

In [12]:
def conditional_probs (corpus, ngram):
    #Pr(w1|w2) = Pr(w1, w2)/Pr(w2)
    n = len(ngram)
    n_less_gram = list()
    i = 0
    for i in range(n-1):
        n_less_gram.append(ngram[i])
    if n > 2:
        n_less = tuple(n_less_gram)
    else:
        n_less = n_less_gram[0]
    n_lap_probs = laplace_smoothing(n, corpus)
    nless_lap_probs = laplace_smoothing(n-1, corpus)
    
    cond_prob = 0.000000001
    numerator = 0.000000001
    for (n_gram, val) in n_lap_probs:
        if ngram == n_gram:
            numerator = val
            break

    denominator = 100000000
    for n_gram, value in nless_lap_probs:
        if n_less == n_gram:
            denominator = value
            break    
    
    return numerator/denominator

def grammar_score(sentence, n):
    # create n-grams from sentence
    size = len(sentence[0])
    n_grams = get_ngrams(n, sentence)
    score = 0
    for ngram in n_grams:
        if score != 0:
            score *= float(conditional_probs(corpus, ngram))
        else:
            score = float(conditional_probs(corpus, ngram))
    # normalize_score based on length
    # total number of sentences of that length = V^size for size > 3 and V^size/2 for size <= 3
    # so the score should represent how likely it is, by dividing the score by some function of the size 
    # which is an accurate representation of the probability of that sentence
    return score

test = "This is a lovely man."
 
cleaned_test = clean(test)
split_test = sentencizer(cleaned_test)
untokenized_test = []
for sentence in split_test:
    untokenized_test.append([sentence])    
test_corpus = tokenize (untokenized_test)

grammar_score (test_corpus, 3)
#print(len(test_corpus[0]))

3.591173812634585e-15

# Part 4 : Spelling Test

In [15]:
def spelling_score(word, n):
    # create n-grams from sentence
    #size = len(word[0])
    n_grams = get_ngrams(n, [word])
    score = 0
    for ngram in n_grams:
        if score != 0:
            score *= float(conditional_probs(characterized_corpus, ngram))
        else:
            score = float(conditional_probs(characterized_corpus, ngram))
    # normalize_score based on length
    # total number of sentences of that length = V^size for size > 3 and V^size/2 for size <= 3
    # so the score should represent how likely it is, by dividing the score by some function of the size 
    # which is an accurate representation of the probability of that sentence
    return score

test = "This"
 
cleaned_test = clean(test)
characterized_test_corpus = characterizer(cleaned_test)

spelling_score(characterized_test_corpus, 3)

0.05558790844239914